#######################################################################################
# GROUP PROJECT 2 - ETL  
## Students:  James Dietz  |  Rafael Santos
### DATA ANALYTICS AND VISUALIZATION COHORT 3
#######################################################################################

## Summary:

The focus on the project was on Extracting, Transforming, and Loading (ETL) international data pertaining to agricultural land resources, cereal crop yields, and cereal crop commodity markets and prices.  The data sources came from the World Bank and the Kaggle.  Each data source was organized by country level data (more details are provided in the Transform Section).
 
We believe that the combined ETL data source could be useful in international comparative analyses of the agricultural resources, cereal crop yields, and price levels.  Data sources varied in the year of most recent available.  We chose to use the most recent year available, 2016, that was common to all datasets:

World Bank URL:  https://data.worldbank.org/topic/agriculture-and-rural-development

Kaggle URL: https://www.kaggle.com/jboysen/global-food-prices/version/1


We took three major steps to build the database:

1) Extract: 
* downloaded the datasets available in CSV files
* reviewed the data using Pandas to identify meaningful fields for the purpose of this project.
 
2) Transform:
* extracted the meaningful fields into dataframes to manipulate the data.
* created and/or modified dataframe indexes to be set as country code IDs (using ISO A3 standard).
* merged dataframes
* created final tables to load into SQL

  AND/OR

* merged loaded tables within SQL enviroment

3) Load:
* manually created the database schema in MySQL
* using SQL Alchemy:
 * created the connection between Pandas (Python) environment and the SQL database.
 * loaded the Pandas dataframe data into the SQL database.
 
The details, including the results obtained, are either described or provided below along with the code used.

In [17]:
import pandas as pd
from sqlalchemy import create_engine

In [18]:
#used for Global Food Price only
import numpy as np

### 1) EXTRACT:  WORLD BANK (WB)

The data were downloaded from the above WB website.  Two datasets were downloaded—(1) cereal crop yields per country in KG per hectacre [cereal_yield], and (2) percent of total country land area that is devoted to agriculture by country [agriculture_land].

Data sets were downloaded as .csv files and were read into pandas via this Jupyter Notebook.  

### 1) EXTRACT:  WB DATA - Cereal Yield

In [19]:
# Loaded data from CSV file into Pandas Dataframe
cereal_yield = "cereal_yield.csv"
cereal_yield_raw_df = pd.read_csv(cereal_yield)
cereal_yield_raw_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,Cereal yield (kg per hectare),AG.YLD.CREL.KG,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,Cereal yield (kg per hectare),AG.YLD.CREL.KG,NaN,1115.1,1079.0,985.8,1082.8,1098.9,...,2040.7,2011.1,1659.9,2029.6,2048.5,2017.5,2133.0,1981.6,2024.7,NaN
2,Angola,AGO,Cereal yield (kg per hectare),AG.YLD.CREL.KG,NaN,828.0,830.3,798.4,875.8,932.0,...,571.4,629.3,662.4,552.0,814.4,888.3,982.0,934.1,904.5,NaN
3,Albania,ALB,Cereal yield (kg per hectare),AG.YLD.CREL.KG,NaN,845.2,941.8,982.3,1023.3,1037.3,...,4315.3,4761.8,4750.7,4883.8,4949.8,4892.6,4873.8,4716.4,4813.0,NaN
4,Andorra,AND,Cereal yield (kg per hectare),AG.YLD.CREL.KG,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2) TRANSFORM: WORLD BANK (WB)

The data sets were made into two separate dataframes (DFs) in the step above, and then were transformed by deleting extraneous columns, which including a number of indicator codes unique to WB data systems and data for years not used in this project.  

### 2) TRANSFORM:  WB DATA - Cereal Yield

In [20]:
cereal_yield_raw_df.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018'],
      dtype='object')

In [21]:
# Create a filtered dataframe from specific columns
cereal_cols = ["Country Name", "Country Code", "2016"]
cereal_yield_df= cereal_yield_raw_df[cereal_cols].copy()

# Rename the column headers
cereal_yield_df = cereal_yield_df.rename(columns={"Country Name": "country_name",
                                                        "Country Code": "country_code",
                                                         "2016": "yield"})

# Clean the data by dropping duplicates and setting the index
#premise_transformed.drop_duplicates("id", inplace=True)
cereal_yield_df.set_index("country_code", inplace=True)

cereal_yield_df

,country_name,yield
country_code,,
ABW,Aruba,NaN
AFG,Afghanistan,1981.600000
AGO,Angola,934.100000
ALB,Albania,4716.400000
AND,Andorra,NaN
ARB,Arab World,1744.945443
ARE,United Arab Emirates,28130.100000
ARG,Argentina,5203.500000
ARM,Armenia,3076.100000


### 2) EXTRACT:  WB DATA - Percentage of Agriculture Land

In [22]:
agriculture_land = "agriculture_land.csv"
agriculture_land_raw_df = pd.read_csv(agriculture_land)
agriculture_land_raw_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,Agricultural land (% of land area),AG.LND.AGRI.ZS,NaN,11.111111,11.111111,11.111111,11.111111,11.111111,...,11.111111,11.111111,11.111111,11.111111,11.111111,11.111111,11.111111,11.111111,NaN,NaN
1,Afghanistan,AFG,Agricultural land (% of land area),AG.LND.AGRI.ZS,NaN,57.745918,57.837821,57.914407,58.010906,58.013969,...,58.067580,58.069111,58.067580,58.067580,58.067580,58.067580,58.067580,58.067580,NaN,NaN
2,Angola,AGO,Agricultural land (% of land area),AG.LND.AGRI.ZS,NaN,45.857063,45.881126,45.897169,45.921232,45.937274,...,46.755434,46.835646,47.316917,47.316917,47.477340,47.477340,47.477340,47.477340,NaN,NaN
3,Albania,ALB,Agricultural land (% of land area),AG.LND.AGRI.ZS,NaN,44.963504,44.963504,45.000000,44.890511,45.145985,...,43.843067,43.843067,43.832117,43.843067,43.332119,42.857301,42.857666,43.127735,NaN,NaN
4,Andorra,AND,Agricultural land (% of land area),AG.LND.AGRI.ZS,NaN,55.319149,55.319149,55.319149,55.319149,55.319149,...,44.191490,42.063831,42.063831,39.914894,40.021275,40.021275,40.021275,39.957448,NaN,NaN


### 2) TRANSFORM:  WB DATA - Percentage of Agriculture Land

In [23]:
agriculture_land_raw_df.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018'],
      dtype='object')

In [24]:
# Create a filtered dataframe from specific columns
ag_cols = ["Country Name", "Country Code", "2016"]
agriculture_land_df= agriculture_land_raw_df[ag_cols].copy()

# Rename the column headers
agriculture_land_df = agriculture_land_df.rename(columns={"Country Name": "country_name",
                                                        "Country Code": "country_code",
                                                         "2016": "percent_land"})

# Clean the data by dropping duplicates and setting the index
#premise_transformed.drop_duplicates("id", inplace=True)
agriculture_land_df.set_index("country_code", inplace=True)

agriculture_land_df

,country_name,percent_land
country_code,,
ABW,Aruba,11.111111
AFG,Afghanistan,58.067580
AGO,Angola,47.477340
ALB,Albania,43.127735
AND,Andorra,39.957448
ARB,Arab World,36.610850
ARE,United Arab Emirates,5.474514
ARG,Argentina,54.335712
ARM,Armenia,58.897086


### 2) TRANSFORM:  WORLD BANK DATA - Merging datasets into a single dataframe table

In [25]:
cereal_df = cereal_yield_df.merge(agriculture_land_df, left_on='country_code', right_on='country_code')
cereal_df

,country_name_x,yield,country_name_y,percent_land
country_code,,,,
ABW,Aruba,NaN,Aruba,11.111111
AFG,Afghanistan,1981.600000,Afghanistan,58.067580
AGO,Angola,934.100000,Angola,47.477340
ALB,Albania,4716.400000,Albania,43.127735
AND,Andorra,NaN,Andorra,39.957448
ARB,Arab World,1744.945443,Arab World,36.610850
ARE,United Arab Emirates,28130.100000,United Arab Emirates,5.474514
ARG,Argentina,5203.500000,Argentina,54.335712
ARM,Armenia,3076.100000,Armenia,58.897086


### 2) TRANSFORM:  WORLD BANK DATA - Cleaning up

In [26]:
cereal_df.drop(columns=['country_name_y'], inplace=True)

In [27]:
cereal_df.rename(index=str, columns={"country_name_x": "country_name"})

,country_name,yield,percent_land
country_code,,,
ABW,Aruba,NaN,11.111111
AFG,Afghanistan,1981.600000,58.067580
AGO,Angola,934.100000,47.477340
ALB,Albania,4716.400000,43.127735
AND,Andorra,NaN,39.957448
ARB,Arab World,1744.945443,36.610850
ARE,United Arab Emirates,28130.100000,5.474514
ARG,Argentina,5203.500000,54.335712
ARM,Armenia,3076.100000,58.897086


In [28]:
agriculture_land_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 264 entries, ABW to ZWE
Data columns (total 2 columns):
country_name    264 non-null object
percent_land    254 non-null float64
dtypes: float64(1), object(1)
memory usage: 6.2+ KB


##  GLOBAL FOOD PRICE

### 1) EXTRACT: GLOBAL FOOD PRICE 

* Imported the country code list from a previous project, as a CSV file
* Downloaded from Kaggle website the CSV file for Global Food Price.

In [29]:
#Converting files from CSV to Pandas dataframe
countrycodes_file = "CountryCodesLookup.csv"
countrycodes = pd.read_csv(countrycodes_file, encoding="ISO-8859-1")
countrycodes = countrycodes.rename(columns={'COUNTRY':'country'})

food_price_file = "global-food-prices/wfp_market_food_prices.csv"
food_price = pd.read_csv(food_price_file, encoding="ISO-8859-1")
food_price.head()

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,1,2014,50.0,WFP
1,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,2,2014,50.0,WFP
2,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,3,2014,50.0,WFP
3,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,4,2014,50.0,WFP
4,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,5,2014,50.0,WFP


### 1) EXTRACT: GLOBAL FOOD PRICE
### - Filtering for interested year (2016).

In [30]:
#Filtering data for the year 2016
food_price2016 = food_price.loc[food_price['mp_year']==2016,\
                ['adm0_name','cur_name','cm_name','pt_name','um_name','mp_month','mp_year','mp_price']]
food_price2016.head()

,adm0_name,cur_name,cm_name,pt_name,um_name,mp_month,mp_year,mp_price
22,Afghanistan,AFN,Bread,Retail,KG,1,2016,55.5
23,Afghanistan,AFN,Bread,Retail,KG,2,2016,55.5
24,Afghanistan,AFN,Bread,Retail,KG,3,2016,55.5
25,Afghanistan,AFN,Bread,Retail,KG,4,2016,50.0
26,Afghanistan,AFN,Bread,Retail,KG,5,2016,50.0


### 2) TRANSFORM:  GLOBAL FOOD PRICE 
### - Cleaning up dataset upfront for improved readability of the code and data itself.

In [31]:
#Adjusting table headers and removing columns

table = pd.pivot_table(food_price2016, values=['mp_price'], \
                               index = ['adm0_name','cur_name','cm_name','um_name'],\
                               aggfunc={"mp_price": np.mean})

table = table.reset_index(level=['um_name'])  #convert pivot table indexes into columns
table = table.reset_index(level=['cm_name'])  #convert pivot table indexes into columns
table = table.reset_index(level=['cur_name'])  #convert pivot table indexes into columns
table = table.reset_index(level=['adm0_name'])  #convert pivot table indexes into columns

#Column names copied from the description provided at https://www.kaggle.com/jboysen/global-food-prices/version/1
table = table.rename(columns={'adm0_name':'country'})
table = table.rename(columns={'cur_name':'currency_ID'})
table = table.rename(columns={'cm_name':'commodity_purchased'})
table = table.rename(columns={'um_name':'unit_of_goods_measurement'})
table = table.rename(columns={'mp_price':'monthly_average_price_paid'})

table.head()

,country,currency_ID,commodity_purchased,unit_of_goods_measurement,monthly_average_price_paid
0,Afghanistan,AFN,Bread,KG,44.749375
1,Afghanistan,AFN,Exchange rate,USD/LCU,67.790319
2,Afghanistan,AFN,Fuel (diesel),L,36.969922
3,Afghanistan,AFN,Rice (low quality),KG,41.392708
4,Afghanistan,AFN,"Wage (non-qualified labour, non-agricultural)",Day,285.026042


### 2) TRANSFORM: GLOBAL FOOD PRICE

Note: An initial filter (using "df.LOC") for each food item by its name returned very few values. Reviewing the data, I noticed two meaningful details:

  1) there were descriptions for more than one type of the same food
  
  2) the unit measures were in some cases different.

The decision was at this point was to keep the food item with highest price, and filter for all that rows containing (instead of equal) to the food item researched.

#### Bread
* Defined criteria and filtered data for the first food item chosen: bread
* Merged the datasets in order to have standard Country ID associated to the table.
* Cleaned up the data for irrelevant columns

In [32]:
#filtering table for all rows with the commodity
table_bread = table[table.commodity_purchased.str.contains('Bread', case=False)]

#sorting the list
table_bread = table_bread.sort_values(by=['country','monthly_average_price_paid'],ascending=True,na_position='first')

#dropping duplicated, keeping item with highest price (=last)
table_bread = table_bread.drop_duplicates(subset='country',keep='last')

#Merging country codes
table_bread = pd.merge(countrycodes, table_bread, how="left", left_on=['country'],right_on=['country'])
table_bread = table_bread.drop(['A2 (ISO)','NUM (UN)','DIALING CODE','spotifyFlag',],axis=1)

table_bread.head()

,country,A3 (UN),currency_ID,commodity_purchased,unit_of_goods_measurement,monthly_average_price_paid
0,Afghanistan,AFG,AFN,Bread,KG,44.749375
1,Albania,ALB,NaN,NaN,NaN,NaN
2,Algeria,DZA,DZD,Bread,Unit,10.000000
3,American Samoa,ASM,NaN,NaN,NaN,NaN
4,Andorra,AND,NaN,NaN,NaN,NaN


#### Wheat
* Defined criteria and filtered data for the first food item chosen: wheat
* Merged the datasets in order to have standard Country ID associated to the table.
* Cleaned up the data for irrelevant columns

In [33]:
#filtering table for all rows with the commodity
table_wheat = table[table.commodity_purchased.str.contains('Wheat', case=False)]

#sorting the list
table_wheat = table_wheat.sort_values(by=['country','monthly_average_price_paid'],ascending=True,na_position='first')

#dropping duplicated, keeping item with highest price (=last)
table_wheat = table_wheat.drop_duplicates(subset='country',keep='last')

#Merging country codes
table_wheat = pd.merge(countrycodes, table_wheat, how="left", left_on=['country'],right_on=['country'])
table_wheat = table_wheat.drop(['A2 (ISO)','NUM (UN)','DIALING CODE','spotifyFlag',],axis=1)

table_wheat.head()

,country,A3 (UN),currency_ID,commodity_purchased,unit_of_goods_measurement,monthly_average_price_paid
0,Afghanistan,AFG,AFN,Wheat,KG,24.075078
1,Albania,ALB,NaN,NaN,NaN,NaN
2,Algeria,DZA,NaN,NaN,NaN,NaN
3,American Samoa,ASM,NaN,NaN,NaN,NaN
4,Andorra,AND,NaN,NaN,NaN,NaN


#### Rice
* Defined criteria and filtered data for the first food item chosen: rice
* Merged the datasets in order to have standard Country ID associated to the table.
* Cleaned up the data for irrelevant columns

In [34]:
#filtering table for all rows with the commodity
table_rice = table[table.commodity_purchased.str.contains('Rice', case=False)]

#sorting the list
table_rice = table_rice.sort_values(by=['country','monthly_average_price_paid'],ascending=True,na_position='first')

#removing items different from KG
#table_rice=table_rice.loc[table['unit_of_goods_measurement']=="KG",\
#                      ['country','currency_ID','commodity_purchased','unit_of_goods_measurement','monthly_average_price_paid']]

#dropping duplicated, keeping item with highest price (=last)
table_rice = table_rice.drop_duplicates(subset='country',keep='last')

#Merging country codes
table_rice = pd.merge(countrycodes, table_rice, how="left", left_on=['country'],right_on=['country'])
table_rice = table_rice.drop(['A2 (ISO)','NUM (UN)','DIALING CODE','spotifyFlag',],axis=1)

table_rice.head()

,country,A3 (UN),currency_ID,commodity_purchased,unit_of_goods_measurement,monthly_average_price_paid
0,Afghanistan,AFG,AFN,Rice (low quality),KG,41.392708
1,Albania,ALB,NaN,NaN,NaN,NaN
2,Algeria,DZA,DZD,Rice,KG,106.250000
3,American Samoa,ASM,NaN,NaN,NaN,NaN
4,Andorra,AND,NaN,NaN,NaN,NaN


### 2) TRANSFORM: GLOBAL FOOD PRICE - Main table creation

* Merge the three tables above into a single one.
* Clean up data for duplicate columns.
* Rename columns for compliance with SQL language standard for header name.
* Standardize index column by Country ID.

In [35]:
#Merging 3 tables into one.
table_food = pd.merge(table_bread, table_rice, how="left", on=['country'],suffixes=('','y'))
table_food = table_food.drop(['A3 (UN)y','currency_IDy'],axis=1)

table_food = table_food.rename(columns={'commodity_purchased':'commodity_purchased_bread',\
                                        'unit_of_goods_measurement':'unit_of_goods_measurement_bread',\
                                        'monthly_average_price_paid':'monthly_average_price_paid_bread',\
                                        'commodity_purchasedy':'commodity_purchased_rice',\
                                        'unit_of_goods_measurementy':'unit_of_goods_measurement_rice',\
                                        'monthly_average_price_paidy':'monthly_average_price_paid_rice'})

table_food = pd.merge(table_food, table_wheat, how="left", on=['country'],suffixes=('','y'))
table_food = table_food.drop(['A3 (UN)y','currency_IDy'],axis=1)

table_food = table_food.rename(columns={'commodity_purchased':'commodity_purchased_wheat',\
                                        'unit_of_goods_measurement':'unit_of_goods_measurement_wheat',\
                                        'monthly_average_price_paid':'monthly_average_price_paid_wheat'})

table_food = table_food.rename(columns={"A3 (UN)": "country_code"})

In [36]:
table_food

,country,country_code,currency_ID,commodity_purchased_bread,unit_of_goods_measurement_bread,monthly_average_price_paid_bread,commodity_purchased_rice,unit_of_goods_measurement_rice,monthly_average_price_paid_rice,commodity_purchased_wheat,unit_of_goods_measurement_wheat,monthly_average_price_paid_wheat
0,Afghanistan,AFG,AFN,Bread,KG,44.749375,Rice (low quality),KG,41.392708,Wheat,KG,24.075078
1,Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Algeria,DZA,DZD,Bread,Unit,10.000000,Rice,KG,106.250000,NaN,NaN,NaN
3,American Samoa,ASM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Anguilla,AIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Antarctica,ATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Antigua and Barbuda,ATG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Argentina,ARG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
table_food.set_index("country_code", inplace=True)

In [38]:
table_food

,country,currency_ID,commodity_purchased_bread,unit_of_goods_measurement_bread,monthly_average_price_paid_bread,commodity_purchased_rice,unit_of_goods_measurement_rice,monthly_average_price_paid_rice,commodity_purchased_wheat,unit_of_goods_measurement_wheat,monthly_average_price_paid_wheat
country_code,,,,,,,,,,,
AFG,Afghanistan,AFN,Bread,KG,44.749375,Rice (low quality),KG,41.392708,Wheat,KG,24.075078
ALB,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DZA,Algeria,DZD,Bread,Unit,10.000000,Rice,KG,106.250000,NaN,NaN,NaN
ASM,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AND,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AGO,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AIA,Anguilla,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ATA,Antarctica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ATG,Antigua and Barbuda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Note: 

A decision was made to not drop null values, assuming this database structure can be updated in future with refreshed data. It is to say, in case data misssing today for a given country becomes available, it is would be easier to update the field for that country rather than create a entire new row or column. Or ultimately, it is also easy to drop Null values in other analysis.

### 2) TRANSFORM: GLOBAL FOOD PRICE
### At this step, these columns were manually created in the schema for the SQL database tables.

In [39]:
table_food.columns


Index(['country', 'currency_ID', 'commodity_purchased_bread',
       'unit_of_goods_measurement_bread', 'monthly_average_price_paid_bread',
       'commodity_purchased_rice', 'unit_of_goods_measurement_rice',
       'monthly_average_price_paid_rice', 'commodity_purchased_wheat',
       'unit_of_goods_measurement_wheat', 'monthly_average_price_paid_wheat'],
      dtype='object')

### 3) LOAD 

#### Comment on additional work done in SQL 

The pandas dataframes were loaded into MYSQL using SQL Alchemy (establishing a connection between Jupyter NB and MYSQL).  A MYSQL database (agriculture_db) was created and three tables were created—cereal_yield, agiculture_land, and table_food.

An inner join was written selecting each of the relevant columns and joining the two WB tables on the common column of country code [country_code] and then joining that with the table_food table.  It should be noted that this resulted in many null values in the columns.  Various joins (right, left, inner) in MYSQL were performed as examples that demonstrates that exact type of join will affect the number of rows of the table and are dependent on the data analyses that might be employed.

### 3) LOAD: CONNECTING TO SQL DB

### At this point, the database was created in SQL. See the SQL query used in Git Hub
#### https://github.com/messagetorafael/groupproject2/blob/master/agriculture.sql




In [41]:
#change root to my mysql user name and my sql password
rds_connection_string = "root:[suppressed]@localhost/agriculture_db"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')

In [42]:
# confirm the tables were created successfully
engine.table_names()

['agriculture_land', 'cereal_yield', 'table_food']

### 3) LOAD: LOAD DATA TO SQL Database tables



#### WORLD BANK - Issues found 
The two main dataframes were loaded into tables created in MYSQL (as indicated above).  However, when they were loaded a Transform issue arose. The datatype in percent of land devoted to agriculture in pandas was a float of  2 digits plus 6 decimal places (i.e., a percentage with a lot of decimal places).

Because the MYSQL table column was original designated as “decimal” these decimal place values were truncated.  For example, 11.111111 was rendered 11 percent in SQL.  As a result, we had to update the MYSQL table column as DECIMAL (8,6) which preserved the data.

In [43]:
cereal_yield_df.to_sql(name='cereal_yield', con=engine, if_exists='append', index=True) 

In [44]:
agriculture_land_df.to_sql(name='agriculture_land', con=engine, if_exists='append', index=True) 

In [45]:
table_food.to_sql(name='table_food', con=engine, if_exists='append', index=True) 

C:\Users\BahMN\Anaconda3\envs\PythonWebMongo\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'monthly_average_price_paid_bread' at row 1")
  result = self._query(query)
C:\Users\BahMN\Anaconda3\envs\PythonWebMongo\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'monthly_average_price_paid_rice' at row 1")
  result = self._query(query)
C:\Users\BahMN\Anaconda3\envs\PythonWebMongo\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'monthly_average_price_paid_wheat' at row 1")
  result = self._query(query)
C:\Users\BahMN\Anaconda3\envs\PythonWebMongo\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'monthly_average_price_paid_bread' at row 11")
  result = self._query(query)
C:\Users\BahMN\Anaconda3\envs\PythonWebMongo\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'monthly_average_price_paid_rice' at row 11")
